In [13]:
"""
https://leetcode.com/problems/traffic-light-controlled-intersection/
https://leetcode.ca/2019-06-01-1279-Traffic-Light-Controlled-Intersection/


There is an intersection of two roads.
First road is road A where cars travel from North to South in direction 1 and from South to North in direction 2.
Second road is road B where cars travel from West to East in direction 3 and from East to West in direction 4.

dir 1   A A A
  |     A A A  <- dir 4
  V     A A A
B B B B       B B B B 
B B B B       B B B B 
B B B B       B B B B 
        A A A    ^
  ->    A A A    |
 dir 3  A A A  dir 2

There is a traffic light located on each road before the intersection.
A traffic light can either be green or red.

Green means cars can cross the intersection in both directions of the road.
Red means cars in both directions cannot cross the intersection and must wait until the light turns green.
The traffic lights cannot be green on both roads at the same time. 
That means when the light is green on road A, it is red on road B and when the light is green on road B, it is red on road A.

Initially, the traffic light is green on road A and red on road B. 
When the light is green on one road, all cars can cross the intersection in both directions until the light becomes green on the other road. 
No two cars traveling on different roads should cross at the same time.

Design a deadlock-free traffic light controlled system at this intersection.

Implement the function void carArrived(carId, roadId, direction, turnGreen, crossCar) where:

carId is the id of the car that arrived.
roadId is the id of the road that the car travels on.
direction is the direction of the car.
turnGreen is a function you can call to turn the traffic light to green on the current road.
crossCar is a function you can call to let the current car cross the intersection.
Your answer is considered correct if it avoids cars deadlock in the intersection. 
Turning the light green on a road when it was already green is considered a wrong answer.


Constraints:

1 <= cars.length <= 20
cars.length = directions.length
cars.length = arrivalTimes.length
All values of cars are unique
1 <= directions[i] <= 4
arrivalTimes is non-decreasing
"""
import threading
import time

class TrafficLight:
    def __init__(self):
        self.lock = threading.Lock()
        self.road = 1 # 1 (dir 1, 2) or 2 (dir 3, 4)
    
    def carArrived(self, carId, roadId, direction, turnGreen, crossCar):
        self.lock.acquire()
        if self.road != roadId:
            self.road = roadId
            turnGreen()
        crossCar()
        self.lock.release()


def test(cars, directions, arrivalTimes):
    light = TrafficLight()
    idx = 0
    threads = []
    for t in range(1, arrivalTimes[-1]+1):
        time.sleep(0.5)
        while idx < len(arrivalTimes) and t >= arrivalTimes[idx]:
            carId = cars[idx]
            roadId = (directions[idx]-1)//2+1
            direction = directions[idx]
            turnGreen = lambda: print("Traffic Light On Road", "A" if roadId == 1 else "B", "Is Green")
            crossCar = lambda: print("Car", carId, "Has Passed Road", "A" if roadId == 1 else "B", "In Direction", direction)
            car = threading.Thread(target=light.carArrived, args=[carId,
                                                                roadId,
                                                                direction,
                                                                turnGreen,
                                                                crossCar])
            car.start()
            threads.append(car)
            idx += 1

    for car in threads:
        car.join()
    print()

test([1,3,5,2,4], [2,1,2,4,3], [10,20,30,40,50])
# "Car 1 Has Passed Road A In Direction 2",    #// Traffic light on road A is green, car 1 can cross the intersection.
# "Car 3 Has Passed Road A In Direction 1",    #// Car 3 crosses the intersection as the light is still green.
# "Car 5 Has Passed Road A In Direction 2",    #// Car 5 crosses the intersection as the light is still green.
# "Traffic Light On Road B Is Green",          #// Car 2 requests green light for road B.
# "Car 2 Has Passed Road B In Direction 4",    #// Car 2 crosses as the light is green on road B now.
# "Car 4 Has Passed Road B In Direction 3"     #// Car 4 crosses the intersection as the light is still green.

test([1,2,3,4,5], [2,4,3,3,1], [10,20,30,40,40])
# "Car 1 Has Passed Road A In Direction 2",    #// Traffic light on road A is green, car 1 can cross the intersection.
# "Traffic Light On Road B Is Green",          #// Car 2 requests green light for road B.
# "Car 2 Has Passed Road B In Direction 4",    #// Car 2 crosses as the light is green on road B now.
# "Car 3 Has Passed Road B In Direction 3",    #// Car 3 crosses as the light is green on road B now.
# "Traffic Light On Road A Is Green",          #// Car 5 requests green light for road A.
# "Car 5 Has Passed Road A In Direction 1",    #// Car 5 crosses as the light is green on road A now.
# "Traffic Light On Road B Is Green",          #// Car 4 requests green light for road B. 
#                                              #   Car 4 blocked until car 5 crosses and then traffic light is green on road B.
# "Car 4 Has Passed Road B In Direction 3"     #// Car 4 crosses as the light is green on road B now.
#
#This is a dead-lock free scenario. 
# Note that the scenario when car 4 crosses before turning light into green on road A and allowing car 5 to pass is also correct 
# and Accepted scenario.


Car 1 Has Passed Road A In Direction 2
Car 3 Has Passed Road A In Direction 1
Car 5 Has Passed Road A In Direction 2
Traffic Light On Road B Is Green
Car 2 Has Passed Road B In Direction 4
Car 4 Has Passed Road B In Direction 3

Car 1 Has Passed Road A In Direction 2
Traffic Light On Road B Is Green
Car 2 Has Passed Road B In Direction 4
Car 3 Has Passed Road B In Direction 3
Car 4 Has Passed Road B In Direction 3
Traffic Light On Road A Is Green
Car 5 Has Passed Road A In Direction 1

